In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")

time.sleep(5)  # JS 로딩 대기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify()[4000:]) 


  
	   	var menuItemSelectHandler = function(event) {
	  		contextMenuHandler(event.menuItemCaption);
	   	};

	   	var layoutCompleteHandler = function(event) {
	   		dataGrid = gridRoot.getDataGrid();
	   		dataGrid.addEventListener("menuItemSelect", menuItemSelectHandler);
	   		dataGrid.addEventListener("itemClick", getDataJson);		//셀 클릭 이벤트
	   		
	   		imageColumn = gridRoot.getObjectById("dg1col19");
	        imageColumn.addEventListener("itemImageClick", getDataJson);	//이미지 클릭 이벤트
	   		
	   	};
	   	gridRoot.addEventListener("layoutComplete", layoutCompleteHandler);    	
	};
	jsVars += "&rMateOnLoadCallFunction=gridReadyHandler";
	// ----------------------- rMateGrid 설정 끝 -----------------------
	
	
	
	//
	function selLicClsCd(licCls){
	    var url  = '/web/TG/TG200/TG2100/AJAX/LicCls_AJAX.jsp';
        var params ="selLicCls="+licCls;
        sendRequest(url,params,selectLicClsState,"POST");       
    }
    
    //Callback함수 호출
    function selectLicClsState(){
        if(xm

In [12]:

#다시 돌아와야하는 구조....

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException
import time
import os

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"

options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업/다운로드 함수 ----------------------
def wait_for_popup(driver, timeout=10):
    WebDriverWait(driver, timeout).until(lambda d: len(d.window_handles) > 1)

def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            return True
        time.sleep(1)
    return False

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0

    while retries < 2:  # 최대 2번 시도
        try:
            wait_for_popup(driver)
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()
            time.sleep(3)

            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            if wait_for_download_complete():
                print(f"[✅ 성공] {location_name}")
                break
            else:
                print(f"[⚠️ 타임아웃] {location_name}")

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            time.sleep(2)
        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)

    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")

# ---------------------- 현재 보이는 첫 번째 버튼 클릭 ----------------------
def click_visible_button(driver):
    """
    현재 내부 스크롤 상태에서 가장 위에 보이는 버튼을 클릭
    """
    return driver.execute_script("""
        const container = document.querySelector('#rMateH5__Content69');
        if (!container) return null;

        const buttons = container.querySelectorAll('img');
        for (let btn of buttons) {
            const rect = btn.getBoundingClientRect();
            const cRect = container.getBoundingClientRect();
            if (rect.top >= cRect.top && rect.bottom <= cRect.bottom) {
                btn.click();
                return true;
            }
        }
        return null;
    """)

# ---------------------- 내부 스크롤 + 순차 다운로드 ----------------------
def download_incremental(driver, wait, year_value, sido_text, jijace_text):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")
    downloaded = 0

    while True:
        clicked = click_visible_button(driver)
        if not clicked:
            print("📌 보이는 버튼 없음 → 종료")
            break

        handle_structure_download(
            driver, wait,
            location_name=f"{year_value}_{sido_text}_{jijace_text}_지점{downloaded+1}"
        )
        downloaded += 1
        time.sleep(1)

        # 내부 스크롤 조금만 내리기
        current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
        max_scroll = driver.execute_script(
            "return arguments[0].scrollHeight - arguments[0].clientHeight;", scroll_element
        )
        if current_scroll >= max_scroll:
            print("📌 내부 스크롤 끝 → 종료")
            break

        driver.execute_script("arguments[0].scrollTop += 50;", scroll_element)
        time.sleep(0.5)

    print(f"✅ 총 {downloaded}개 지점 다운로드 완료")

# ---------------------- 실행 ----------------------
test_years = ["2023"]

for year_value in test_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    sido_select = Select(driver.find_element(By.ID, "sido"))
    sido_select.select_by_index(1)
    sido_text = sido_select.first_selected_option.text

    wait.until(EC.presence_of_element_located((By.ID, "jijace")))
    jijace_select = Select(driver.find_element(By.ID, "jijace"))
    jijace_select.select_by_index(1)
    jijace_text = jijace_select.first_selected_option.text

    driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a").click()

    try:
        alert = driver.switch_to.alert
        alert.accept()
    except NoAlertPresentException:
        pass

    time.sleep(2)
    download_incremental(driver, wait, year_value, sido_text, jijace_text)

driver.quit()


[✅ 성공] 2023_서울_강남구_지점1
[✅ 성공] 2023_서울_강남구_지점2
[❌ 실패] 2023_서울_강남구_지점3 - 'NoneType' object has no attribute 'is_displayed' (재시도 1/2)
[❌ 실패] 2023_서울_강남구_지점3 - Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.169); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff6119ae415+77285]
	GetHandleVerifier [0x0x7ff6119ae470+77376]
	(No symbol) [0x0x7ff611779a6a]
	(No symbol) [0x0x7ff6117641eb]
	(No symbol) [0x0x7ff6117656cf]
	(No symbol) [0x0x7ff61177a4bf]
	(No symbol) [0x0x7ff61175194d]
	(No symbol) [0x0x7ff61180a338]
	(No symbol) [0x0x7ff6117f85d0]
	(No symbol) [0x0x7ff6117c1521]
	(No symbol) [0x0x7ff6117c22b3]
	GetHandleVerifier [0x0x7ff611c91efd+3107021]
	GetHandleVerifier [0x0x7ff611c8c29d+3083373]
	GetHandleVerifier [0x0x7ff611cabedd+3213485]
	

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff6119ae415+77285]
	GetHandleVerifier [0x0x7ff6119ae470+77376]
	(No symbol) [0x0x7ff61177989c]
	(No symbol) [0x0x7ff6117c064f]
	(No symbol) [0x0x7ff6117f84d2]
	(No symbol) [0x0x7ff6117f2fd3]
	(No symbol) [0x0x7ff6117f2099]
	(No symbol) [0x0x7ff611744895]
	GetHandleVerifier [0x0x7ff611c91efd+3107021]
	GetHandleVerifier [0x0x7ff611c8c29d+3083373]
	GetHandleVerifier [0x0x7ff611cabedd+3213485]
	GetHandleVerifier [0x0x7ff6119c884e+184862]
	GetHandleVerifier [0x0x7ff6119d055f+216879]
	(No symbol) [0x0x7ff611743890]
	GetHandleVerifier [0x0x7ff611dc6558+4370216]
	BaseThreadInitThunk [0x0x7ffb61411fe4+20]
	RtlUserThreadStart [0x0x7ffb6385ef91+33]
